# Initial distribution evolution and immutable osservables

**Are we in SWAN?**

In [ ]:
%pip install --user crank-nicolson-numba
%pip install --user joblib
%pip install --user lmfit
%pip install --user tqdm

**Do we want the matplotlib interactive magicness?**

In [1]:
%matplotlib widget

**Library Import**

In [2]:
N_JOBS = 4

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate
from tqdm.notebook import tqdm
import crank_nicolson_numba.generic as cn
import itertools
# For parallelization
from joblib import Parallel, delayed

import nekhoroshev_tools as nt
import expo_tools as et
import poly_tools as pt

from lmfit import minimize, Parameters


# Exponential distribution evolution

In [4]:
def rho_0(I, damping_position=np.nan, l=np.nan):
    if np.isnan(damping_position) or np.isnan(l):
        return np.exp(-I)
    else:
        return np.exp(-I) / (1 + np.exp((I - damping_position)/l))

In [5]:
cn_sampling = 50000
cn_time_steps = 100

In [6]:
I_max = 5.0
I_0 = 4.8

I_star = 10.0
k = 0.33
exponent = 1/(2*k)

I_list, dI = np.linspace(0.0, I_max, cn_sampling, retstep=True)

I_0_index = np.argmin(np.absolute(I_list-I_0))

In [7]:
popt_0, pcov_0 = scipy.optimize.curve_fit(
    lambda x, k: np.exp(-k*x),
    I_list[:I_0_index],
    rho_0(I_list[:I_0_index], I_0, dI*5)
)

In [8]:
plt.figure()
plt.plot(I_list, rho_0(I_list, I_0, dI*5), label="Initial distribution")
plt.plot(I_list[:I_0_index], np.exp(-popt_0[0] * I_list[:I_0_index]), label="Exp fit until $I_0$, $\epsilon$={:.2}".format(popt_0[0]))
plt.legend()
plt.xlabel("$I$")
plt.ylabel("$\\rho$")
plt.title("Initial distribution and (obvious) fit")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Initial distribution and (obvious) fit')

In [9]:
plt.figure()
plt.plot(I_list[:I_0_index], rho_0(I_list, I_0, dI*5)[:I_0_index], label="Initial distribution")
plt.plot(I_list[:I_0_index], np.exp(-popt_0[0] * I_list[:I_0_index]), label="Exp fit until $I_0$, $\epsilon$={:.2}".format(popt_0[0]))
plt.legend()
plt.xlabel("$I$")
plt.ylabel("$\\rho$")
plt.yscale("log")
plt.title("Initial distribution and (obvious) fit")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Initial distribution and (obvious) fit')

In [10]:
c_nek = nt.standard_c(0.0, I_max, I_star, exponent)
dt_nek = nt.current_peak_time(I_0, I_max, I_star, exponent, c_nek)/cn_time_steps
engine_nek = cn.cn_generic(
    0, I_max,
    rho_0(I_list, I_0, dI*5),
    dt_nek,
    lambda x: nt.D(x, I_star, exponent, c_nek, True)
)

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)


In [11]:
data = []
time = []
fit_data = []
fit_data_log = []

engine_nek.reset()

data.append(engine_nek.get_data())
popt, pcov = scipy.optimize.curve_fit(
    lambda x, k: np.exp(-k*x),
    I_list[:I_0_index],
    data[-1][:I_0_index]
)
fit_data.append((popt, pcov))
for i in tqdm(range(cn_time_steps)):
    engine_nek.iterate(1000)
    data.append(engine_nek.get_data())
    popt, pcov = scipy.optimize.curve_fit(
        lambda x, k: (np.exp(-k*x)),
        I_list[:I_0_index],
        (data[-1][:I_0_index])
    )
    fit_data.append((popt, pcov))
    popt, pcov = scipy.optimize.curve_fit(
        lambda x, k: np.log(np.exp(-k*x)),
        I_list[:I_0_index],
        np.log(data[-1][:I_0_index])
    )
    fit_data_log.append((popt, pcov))

  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
plt.figure()
plt.plot([p[0] for p in fit_data])
#plt.plot([p[0] for p in fit_data_log])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plt.figure()
plt.plot(I_list[:I_0_index], data[0][:I_0_index])
plt.plot(I_list[:I_0_index], np.exp(-fit_data[0][0] * I_list[:I_0_index]))
plt.plot(I_list[:I_0_index], data[-1][:I_0_index])
plt.plot(I_list[:I_0_index], np.exp(-fit_data[-1][0] * I_list[:I_0_index]))
#plt.plot(I_list[:I_0_index], np.exp(-fit_data_log[-1][0] * I_list[:I_0_index]))
#plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plt.figure()
plt.plot(I_list[:I_0_index], data[0][:I_0_index])
plt.plot(I_list[:I_0_index], np.exp(-fit_data[0][0] * I_list[:I_0_index]))
plt.plot(I_list[:I_0_index], data[-1][:I_0_index])
plt.plot(I_list[:I_0_index], np.exp(-fit_data[-1][0] * I_list[:I_0_index]))
#plt.plot(I_list[:I_0_index], np.exp(-fit_data_log[-1][0] * I_list[:I_0_index]))
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
popt_0

array([1.00000042])

In [16]:
I_max = 5.0
I_0 = 4.8

I_star = 6.0
k = 0.33
exponent = 1/(2*k)

I_list, dI = np.linspace(0.0, I_max, cn_sampling, retstep=True)

I_0_index = np.argmin(np.absolute(I_list-I_0))

In [17]:
popt_0, pcov_0 = scipy.optimize.curve_fit(
    lambda x, k: np.exp(-k*x),
    I_list[:I_0_index],
    rho_0(I_list[:I_0_index], I_0, dI*5)
)

In [18]:
plt.figure()
plt.plot(I_list, rho_0(I_list, I_0, dI*5), label="Initial distribution")
plt.plot(I_list[:I_0_index], np.exp(-popt_0[0] * I_list[:I_0_index]), label="Exp fit until $I_0$, $\epsilon$={:.2}".format(popt_0[0]))
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
c_nek = nt.standard_c(0.0, I_max, I_star, exponent)
dt_nek = nt.current_peak_time(I_0, I_max, I_star, exponent, c_nek)/cn_time_steps
engine_nek = cn.cn_generic(
    0, I_max,
    rho_0(I_list, I_0, dI*5),
    dt_nek,
    lambda x: nt.D(x, I_star, exponent, c_nek, True)
)

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)


In [20]:
data = []
fit_data = []

engine_nek.reset()

data.append(engine_nek.get_data())
popt, pcov = scipy.optimize.curve_fit(
    lambda x, k: np.exp(-k*x),
    I_list[:I_0_index],
    data[-1][:I_0_index]
)
fit_data.append((popt, pcov))
    
for i in tqdm(range(cn_time_steps)):
    engine_nek.iterate(1000)
    data.append(engine_nek.get_data())
    popt, pcov = scipy.optimize.curve_fit(
        lambda x, k: np.log(np.exp(-k*x)),
        I_list[:I_0_index],
        np.log(data[-1][:I_0_index])
    )
    fit_data.append((popt, pcov))

  0%|          | 0/100 [00:00<?, ?it/s]

In [21]:
plt.figure()
plt.plot([p[0] for p in fit_data])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
plt.figure()
plt.plot(I_list[:I_0_index], data[0][:I_0_index])
plt.plot(I_list[:I_0_index], np.exp(-fit_data[0][0] * I_list[:I_0_index]))
plt.plot(I_list[:I_0_index], data[-1][:I_0_index])
plt.plot(I_list[:I_0_index], np.exp(-fit_data[-1][0] * I_list[:I_0_index]))
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
popt_0

array([1.00000042])